## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-21-13-00-00 +0000,wsj,Serena Williams Is the Surprising New Face of ...,https://www.wsj.com/articles/serena-williams-i...
1,2025-08-21-12-58-51 +0000,bbc,Woman jailed for race hate post released from ...,https://www.bbc.com/news/articles/c5yl7p4l11po...
2,2025-08-21-12-58-27 +0000,nyt,Joao Silva’s Journey Back to Afghanistan,https://www.nytimes.com/2025/08/21/world/asia/...
3,2025-08-21-12-56-42 +0000,startribune,Live at the State Fair: Follow along as we try...,https://www.startribune.com/mn-state-fair-food...
4,2025-08-21-12-52-57 +0000,nyt,A Huge Cave in Kentucky Was a Smelly Disaster....,https://www.nytimes.com/2025/08/20/climate/hor...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2298/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
112,trump,46
3,new,26
191,ukraine,13
86,russia,12
436,fire,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
226,2025-08-20-20-36-51 +0000,nypost,Kremlin’s new demands threaten to derail peace...,https://nypost.com/2025/08/20/us-news/kremlins...,115
157,2025-08-21-01-00-00 +0000,wsj,President Trump and his allies are waging a ca...,https://www.wsj.com/economy/central-banking/fe...,93
254,2025-08-20-18-59-56 +0000,nypost,"Trump plays DJ on White House’s new patio, bla...",https://nypost.com/2025/08/20/us-news/trump-pl...,89
166,2025-08-21-00-19-00 +0000,wsj,President Trump has ordered the Pentagon to se...,https://www.wsj.com/politics/national-security...,89
202,2025-08-20-21-45-00 +0000,wsj,President Trump has called on a Biden-appointe...,https://www.wsj.com/economy/central-banking/fe...,88


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
226,115,2025-08-20-20-36-51 +0000,nypost,Kremlin’s new demands threaten to derail peace...,https://nypost.com/2025/08/20/us-news/kremlins...
157,47,2025-08-21-01-00-00 +0000,wsj,President Trump and his allies are waging a ca...,https://www.wsj.com/economy/central-banking/fe...
11,45,2025-08-21-12-30-03 +0000,nypost,US keeping lofty EU car tariffs until levies o...,https://nypost.com/2025/08/21/us-news/us-keepi...
217,44,2025-08-20-20-57-46 +0000,nypost,"JD Vance, Stephen Miller mock ‘old, white’ pro...",https://nypost.com/2025/08/20/us-news/jd-vance...
333,44,2025-08-20-14-01-00 +0000,wsj,Billionaire Bill Ackman has a new fascination:...,https://www.wsj.com/us-news/education/bill-ack...
72,40,2025-08-21-09-39-40 +0000,bbc,Palestinians flee Gaza City districts as Israe...,https://www.bbc.com/news/articles/clyr7l0z9edo...
108,38,2025-08-21-05-56-59 +0000,nypost,Screaming man who tried to run from federal of...,https://nypost.com/2025/08/21/us-news/dc-migra...
166,38,2025-08-21-00-19-00 +0000,wsj,President Trump has ordered the Pentagon to se...,https://www.wsj.com/politics/national-security...
202,37,2025-08-20-21-45-00 +0000,wsj,President Trump has called on a Biden-appointe...,https://www.wsj.com/economy/central-banking/fe...
266,34,2025-08-20-18-26-00 +0000,wsj,A federal judge on Wednesday denied the Justic...,https://www.wsj.com/politics/policy/jeffrey-ep...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
